In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ./jars/postgresql-42.6.0.jar,./jars/clickhouse-jdbc-0.4.6.jar pyspark-shell'

In [2]:
from pyspark.sql import SparkSession

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Jupyter-Spark") \
    .master("local") \
    .getOrCreate()

# Пример DataFrame
data = [("Alice", 25), ("Bob", 30)]
df = spark.createDataFrame(data, ["Name", "Age"])

# Показываем данные
df.show()

# Останавливаем SparkSession
spark.stop()

25/05/23 22:34:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
+-----+---+



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL") \
    .getOrCreate()

print(spark.sparkContext._jsc.sc().listJars())

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
jdbc_url = "jdbc:postgresql://localhost:5432/db"
properties = {
    "user": "Maxim",
    "password": "12345",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# 6. Останавливаем SparkSession
spark.stop()

List(spark://HONOR-MB15:36969/jars/postgresql-42.6.0.jar, spark://HONOR-MB15:36969/jars/clickhouse-jdbc-0.4.6.jar)
Исходные данные:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Данные, прочитанные из PostgreSQL:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Агрегированные данные (сумма зарплат по департаментам):


,Department,Total_Salary
0,Sales,16500
1,HR,8500


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# 1. Создаем SparkSession
spark = SparkSession.builder \
    .appName("Spark SQL with PostgreSQL and ClickHouse") \
    .getOrCreate()

print(spark.sparkContext._jsc.sc().listJars())

# 2. Создаем DataFrame
data = [
    ("Alice", "Sales", 5000),
    ("Bob", "Sales", 6000),
    ("Charlie", "HR", 4000),
    ("David", "HR", 4500),
    ("Eve", "Sales", 5500)
]
columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data, columns)

# Показываем исходный DataFrame
print("Исходные данные:")
display(df.toPandas())

# 3. Записываем данные в PostgreSQL
pg_jdbc_url = "jdbc:postgresql://localhost:5432/db"
properties = {
    "user": "Maxim",
    "password": "12345",
    "driver": "org.postgresql.Driver"
}

# Записываем DataFrame в таблицу 'employees'
df.write.jdbc(url=pg_jdbc_url, table="employees", mode="overwrite", properties=properties)

# 4. Читаем данные из PostgreSQL
df_from_postgres = spark.read.jdbc(url=pg_jdbc_url, table="employees", properties=properties)

print("Данные, прочитанные из PostgreSQL:")
display(df_from_postgres.toPandas())

# 5. Выполняем агрегацию данных
# Группируем по департаменту и считаем общую зарплату
aggregated_df = df_from_postgres.groupBy("Department").agg(sum("Salary").alias("Total_Salary"))

print("Агрегированные данные (сумма зарплат по департаментам):")
pandas_df = aggregated_df.toPandas()
display(pandas_df)

# Записываем данные в ClickHouse через JDBC
ch_jdbc_url = "jdbc:clickhouse://localhost:8123/default"
properties = {
    "driver": "com.clickhouse.jdbc.ClickHouseDriver",
    "user": "Maxim",
    "password": "12345"
}

# Записываем данные в ClickHouse
aggregated_df.write.jdbc(url=ch_jdbc_url, table="default.report_ch", mode="append", properties=properties)

# Читаем данные из ClickHouse
ch_df = spark.read.jdbc(url=ch_jdbc_url, table="default.report_ch", properties=properties)

# Создаем временное view
ch_df.createOrReplaceTempView("report_ch_view")

ch_pandas_df = spark.sql("select Total_Salary from report_ch_view where Department = 'HR' limit 1").toPandas()

print("Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:")
display(ch_pandas_df)

# 6. Останавливаем SparkSession
spark.stop()

List(spark://HONOR-MB15:38707/jars/postgresql-42.6.0.jar, spark://HONOR-MB15:38707/jars/clickhouse-jdbc-0.4.6.jar)
Исходные данные:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Данные, прочитанные из PostgreSQL:


,Name,Department,Salary
0,Alice,Sales,5000
1,Bob,Sales,6000
2,Charlie,HR,4000
3,David,HR,4500
4,Eve,Sales,5500


Агрегированные данные (сумма зарплат по департаментам):


,Department,Total_Salary
0,Sales,16500
1,HR,8500


25/05/23 22:35:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/23 22:35:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/05/23 22:35:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [1490c22d-24ef-4204-8fcf-3502cd580f68] (2 queries & 0 savepoints) is committed.
25/05/23 22:35:07 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [11844f96-d748-4167-975a-0c138cc684c9] (0 queries & 0 savepoints) is committed.


Агрегированные данные (сумма зарплат по департаментам) из ClickHouse где выбран департамент HR:


,Total_Salary
0,8500
